In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import pandas as pd
from IPython.display import display # データフレーム表示用関数
import seaborn as sns
import pymc as pm
import arviz as az
# 表示オプション調整
# NumPy表示形式の設定
np.set_printoptions(precision=3, floatmode='fixed')
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)
# 方眼表示ON
plt.rcParams['axes.grid'] = True
# データフレームでの表示精度
pd.options.display.float_format = '{:.3f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

## ベルヌーイ分布（`pm.Bernoulli`クラス）

In [ ]:
# Code 2.1 ベルヌーイ分布の確率モデル定義（p=0.5）

# パラメータ設定
p = 0.5

model1 = pm.Model()
with model1:
    # pm.Bernoulli: ベルヌーイ分布
    # p: くじに当たる確率
    x = pm.Bernoulli("x", p=p)

In [ ]:
# Code 2.2 事前分布のサンプリング

with model1:
    prior_samples1 = pm.sample_prior_predictive(random_seed=42)

In [ ]:
# Code 2.3 サンプル値を NumPy 形式で抽出

x_samples1 = prior_samples1["prior"]["x"].values
print(x_samples1)

In [ ]:
# Code 2.4 サンプリング結果の統計分析

summary1 = az.summary(prior_samples1, kind="stats")
display(summary1)

In [ ]:
# Code 2.5 サンプリング結果の可視化

ax = az.plot_dist(x_samples1)
ax.set_title(f"ベルヌーイ分布 p={p}");

## 二項分布（`pm.Binomial`クラス）

In [ ]:
# Code 2.6 二項分布の確率モデル定義（p=0.5, n=5）

# パラメータ設定
p = 0.5
n = 5

model2 = pm.Model()
with model2:
    # pm.Binomial: 二項分布
    # p: くじに当たる確率
    # n: 試行回数
    x = pm.Binomial("x", p=p, n=n)

In [ ]:
# Code 2.7 事前分布のサンプリングと結果分析

with model2:
    # sampling
    prior_samples2 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples2 = prior_samples2["prior"]["x"].values
print(x_samples2)

# サンプリング結果の統計分析
summary2 = az.summary(prior_samples2, kind="stats")
display(summary2)

# サンプリング結果の可視化
ax = az.plot_dist(x_samples2)
ax.set_title(f"二項分布 p={p}, n={n}");

In [ ]:
# Code 2.8 二項分布の確率モデル定義（p=0.5, n=50）

model3 = pm.Model()

with model3:
    x = pm.Binomial("x", p=0.5, n=50)

    prior_samples3 = pm.sample_prior_predictive(random_seed=42)

x_samples3 = prior_samples3["prior"]["x"].values

summary3 = az.summary(prior_samples3, kind="stats")
display(summary3)

ax = az.plot_dist(x_samples2)
ax.set_title(f"二項分布 p=0.5, n=50");

## 正規分布（`pm.Normal`クラス）

In [ ]:
# Code 2.9 アイリス・データセットから、setosa のがく片長の分布を調べる

# データセットの読み込み
df = sns.load_dataset("iris")

# 種類が setosa の行のみ抽出
df1 = df.query('species == "setosa"')

bins = np.arange(4.0, 6.2, 0.2)
sns.histplot(data=df1, x="sepal_length", bins=bins, kde=True)
plt.xticks(bins);


In [ ]:
# Code 2.10 正規分布関数の定義とグラフ描画

def norm(x, mu, sigma):
    return np.exp(-((x - mu)/sigma)**2/2) / (np.sqrt(2 * np.pi) * sigma)

# パラメータ定義
mu1, sigma1 = 3.0, 2.0
mu2, sigma2 = 1.0, 3.0

# グラフ描画用x座標の定義
# 2つの正規分布関数で±3sigmaまで入るように計算
x = np.arange(-8.0, 10.0, 0.01)

# x軸目盛の設定
xticks = np.arange(-8.0, 11.0, 1.0)

# グラフ描画
plt.plot(x, norm(x, mu1, sigma1), label=f"mu={mu1} sigma={sigma1}")
plt.plot(x, norm(x, mu2, sigma2), label=f"mu={mu2} sigma={sigma2}")
plt.xticks(xticks, fontsize=12)
plt.legend(fontsize=12)
plt.title("正規分布関数のグラフ");

In [ ]:
# Code 2.11 正規分布の確率モデル定義（mu=0, sigma=1.0）

# パラメータ設定
mu = 0
sigma = 1.0

model4 = pm.Model()
with model4:
    # pm.Normal: 正規分布
    # mu:平均
    # sigma:標準偏差
    x = pm.Normal("x", mu=mu, sigma=sigma)

In [ ]:
# Code 2.12 サンプリングとサンプリング結果の分析

with model4:
    prior_samples4 = pm.sample_prior_predictive(random_seed=42)

# サンプル値の抽出
x_samples4 = prior_samples4["prior"]["x"].values

# サンプリング結果の可視化
ax = az.plot_dist(x_samples4)
ax.set_title(f"正規分布 mu={mu}, sigma={sigma}");

In [ ]:
# Code 2.13 サンプリング結果のヒストグラム表示

bins = np.arange(-3,3.5,0.5)
ax = az.plot_dist(x_samples4, kind="hist",hist_kwargs={"bins":bins})
plt.xticks(np.arange(-3,4,1))
ax.set_title(f"正規分布 mu={mu}, sigma={sigma}");

In [ ]:
# Code 2.14 サンプリング結果の統計分析

summary4 = az.summary(prior_samples4, kind="stats")
display(summary4)

In [ ]:
# Code 2.15 正規分布の確率モデル定義（mu=3.0, sigma=2.0）とサンプリング結果分析

# パラメータ設定
mu = 3.0
sigma = 2.0

model5 = pm.Model()
with model5:
    # pm.Normal: 正規分布
    # mu:平均
    # sigma:標準偏差
    x = pm.Normal('x', mu=mu, sigma=sigma)

    # サンプリング
    prior_samples5 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples5 = prior_samples5["prior"]["x"].values

# サンプリング結果の統計分析
summary5 = az.summary(prior_samples5, kind='stats')
display(summary5)

# サンプリング結果の可視化
ax = az.plot_dist(x_samples5)
ax.set_title(f"正規分布 mu={mu}, sigma={sigma}");